In [21]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

import datetime
from os import path, environ

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import ipyvolume as ipv
# source file, see docs/5-dataset.md for info on field names
chiller_file = path.join(environ['DATADIR'], 'EngineeringScienceBuilding', 'Chillers.csv')
plot_path = path.join('..', 'docs', 'img')

In [2]:
# Read pre-processed data
df = pd.read_csv(chiller_file, index_col='Time', parse_dates=True, dtype=float)

# Evaporative Cooling Model

Modelling the relationship between the cooling done by the cooling tower and the environmental, system, and control inputs.

$$
\begin{align*}
T(t) &= T(0) e^{-\frac{k T_a v_f R}{T_w c_m m} t}
\end{align*}
$$

Where:

* $T(0)$ is the warm entering water temperature `TempCondOut`,
* $T(t)$ is the exiting cool water temperature `TempCondIn`,
* $T_a$ is ambient air temperature `TempAmbient`,
* $v_f$ is avarage fan speed `0.5 * (PerFreqFanA + PerFreqFanB)`,
* $R$ is solar irradiance - constant if assuming shade.
* $T_w$ is wet-bulb temperature `TempWetBulb`,
* $c_m$ is specific mass heat capacity of water,
* $m$ is the mass of water being cooled - constant if assuming steady flow rate.

The model predicts $T(t)$ from all other factors.

## Multi-Layer Perceptron

Assuming steady flow rate, constant irradiance.

In [26]:
# Data
feature_cols = ['TempCondOut', 'PerFreqFanA', 'PerFreqFanB', 'TempAmbient', 'TempWetBulb'] 
X = df.loc[:, feature_cols]
Y = df['TempCondIn']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1)

In [27]:
# Pipeline
scaler = StandardScaler()              # scale to 0 mean and unit variance
regressor = MLPRegressor(hidden_layer_sizes=(20,20), 
                         learning_rate_init=1e-3,
                         verbose=True) # regression model
reg = Pipeline([('scaler', scaler), ('regressor', regressor)])

In [ ]:
reg.fit(Xtrain, Ytrain)
reg.score(Xtest, Ytest)

In [ ]:
# Plotting learned function
